# グラフの生成のソースコード

1. ヒートマップ

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Read the CSV file
file_name = 'p004.csv'
data = pd.read_csv(f'./data/csv_files/{file_name}')

# Convert the 'date' column to datetime format
data['date'] = pd.to_datetime(data['datetime'])

# Extract day of the week from the 'date' column
data['day_of_week'] = data['date'].dt.day_name()

# Define the start and end dates of the week
start_date = '2022-06-19'
end_date = '2022-06-25'

# Filter the data for the specified week
week_data = data[(data['date'] >= start_date) & (data['date'] <= end_date)]

# Pivot the data to have day_of_week as columns, hour as index, and amount as values
pivot_table = pd.pivot_table(week_data, values='amount', index='hour', columns='day_of_week', aggfunc='median')

# Calculate the correlation matrix
correlation_matrix = pivot_table.corr()

# Plotting the correlation heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Heatmap for the Week (med)')
plt.xlabel('Days of the Week')
plt.ylabel('Days of the Week')
plt.xticks(range(7), ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.yticks(range(7), ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.savefig(f'{file_name}_from_{start_date}_heatmap.svg', format='svg')
plt.show()


2. 一週間分の曜日とデバイス

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
file_name = 'p030'
data = pd.read_csv(f'./data/csv_files/{file_name}.csv')

# Convert the 'date' column to datetime format
data['date'] = pd.to_datetime(data['datetime'])

# Extract day of the week from the 'date' column
data['day_of_week'] = data['date'].dt.day_name()

# Define a color map for the days of the week
color_map = {
    'Monday': 'yellow',
    'Tuesday': 'pink',
    'Wednesday': 'green',
    'Thursday': 'orange',
    'Friday': 'blue',
    'Saturday': 'purple',
    'Sunday': 'red'
}

# Group data by week, day of the week, and hour, and calculate the sum of devices
grouped_data = data.groupby([pd.Grouper(key='date', freq='W'), 'day_of_week', 'hour'])['amount'].sum()

# Reset the index of the grouped data
grouped_data = grouped_data.reset_index()

# Get the unique weeks
weeks = grouped_data['date'].dt.strftime('%Y-%m-%d').unique()

# Plotting and saving the data for each week
for week in weeks:
    plt.figure()
    for day in grouped_data['day_of_week'].unique():
        day_data = grouped_data[(grouped_data['date'].dt.strftime('%Y-%m-%d') == week) & (grouped_data['day_of_week'] == day)]
        plt.plot(day_data['hour'], day_data['amount'], label=day, color=color_map[day])

    plt.xlabel('Time (Hour of the day)')
    plt.ylabel('Amount of Devices')
    plt.title(f'Amount of Devices by Time (Week starting from {week})')
    plt.legend()
    plt.savefig(f'{file_name}_week_{week}.svg', format='svg')
    plt.close()


3. 曜日（複数週）とデバイス数の関係

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
data = pd.read_csv('./data/csv_files/p030.csv')

# Convert the 'date' column to datetime format
data['datetime'] = pd.to_datetime(data['datetime'])
day_week = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

for day in range(0, 7):
    day_data = data[data['day_of_week']==day]
    
    # Group data by date and time, and calculate the sum of devices
    grouped_data = day_data.groupby(['datetime', 'hour'])['amount'].sum()
     
    # Reshape the data to have each Sunday as a separate line
    pivot_data = grouped_data.unstack(level=0)
    # Plotting the data
    pivot_data.plot(kind='line', legend=True)
    plt.xlabel('Time')
    plt.ylabel('Amount of Devices')
    plt.title(f'Amount of Devices by Time at p030 ({day_week[day]})')
    plt.savefig(f'p030_{day}.svg', format='svg')
    plt.close()
